In [30]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
import config
from scipy import stats


from sklearn.model_selection import GridSearchCV

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "3_clean")
interm= os.join(root, "data", "2_intermediate")

plots= os.join(root, "code", "plots")

In [5]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [8]:
# files in clean data google drive
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1jxSI_nq32-W9kPHKvQkzuNGJ5aIBD2j2' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: analysis_bbl_noacs.csv, ID: 1h-tDpBSslfzqzsJe1ny_ujDnr68RKry3
Title: analysis_bbl.csv, ID: 1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G
Title: analysis_box.csv, ID: 1Vzjvta98IG9Hg4SfAGcd8JD8FaF3IXwy
Title: borobox_convex_hull.geojson, ID: 1SzBNj3Ba-v9MEwuCkZT9e9iAwVYUqx8r


In [10]:
df_bbl = drive.CreateFile({'id': '1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G'})
df_bbl.GetContentFile('analysis_bbl.csv')  

df= pd.read_csv('analysis_bbl.csv').drop(["Unnamed: 0"], axis = 1)
df.head()

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,bldgclass_A0,bldgclass_A6,bldgclass_A8,bldgclass_CM,bbl,boroct2010,vacate_ind,borough,landuse,bldgclass
0,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,...,0.0,0.0,0.0,0.0,1012090027,1018100,0.0,MN,2.0,C2
1,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,...,0.0,0.0,0.0,0.0,1012090028,1018100,0.0,MN,2.0,C5
2,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,...,0.0,0.0,0.0,0.0,1012090037,1018100,0.0,MN,3.0,D4
3,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,...,0.0,0.0,0.0,0.0,1012090049,1018100,0.0,MN,3.0,D4
4,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,...,0.0,0.0,0.0,0.0,1012090051,1018100,0.0,MN,3.0,D4


In [35]:
print(df.columns)

Index(['lotarea', 'bldgarea', 'numbldgs', 'numfloors', 'unitsres',
       'unitstotal', 'assessland', 'assesstot', 'exempttot', 'yearbuilt',
       ...
       'bbl', 'boroct2010', 'vacate_ind', 'landuse', 'bldgclass', 'borough_BK',
       'borough_BX', 'borough_MN', 'borough_QN', 'borough_SI'],
      dtype='object', length=148)


In [36]:
#df= pd.get_dummies(df, columns=["borough"])
borocols= ["borough_" + x for x in ["BK", "BX", "MN", "QN", "SI"]]

In [37]:
df.head()

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,bbl,boroct2010,vacate_ind,landuse,bldgclass,borough_BK,borough_BX,borough_MN,borough_QN,borough_SI
0,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,...,1012090027,1018100,0.0,2.0,C2,0,0,1,0,0
1,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,...,1012090028,1018100,0.0,2.0,C5,0,0,1,0,0
2,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,...,1012090037,1018100,0.0,3.0,D4,0,0,1,0,0
3,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,...,1012090049,1018100,0.0,3.0,D4,0,0,1,0,0
4,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,...,1012090051,1018100,0.0,3.0,D4,0,0,1,0,0


In [38]:
len(df. index) 

765026

In [39]:
#print(df.nunique())
print(df['vacate_ind'].value_counts())

0.0    762422
1.0      2604
Name: vacate_ind, dtype: int64


In [40]:
# define predictors
df.columns
predictors = ["lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
             "assesstot", "exempttot", "yearbuilt", "holc_AB", "holc_CD", "holc_D",
             "tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio",
             "viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit",
             "AL", "NB", "average_energy_usage", "average_energy_usage_mi", "landuse_cat", "bldgclass_cat"]
predictors_acs= df.columns[df.columns.get_loc("totpop"):df.columns.get_loc("crowdingrenter") + 1].tolist()
predictors_full= predictors + predictors_acs
predictors_full

['lotarea',
 'bldgarea',
 'numbldgs',
 'numfloors',
 'unitsres',
 'unitstotal',
 'assessland',
 'assesstot',
 'exempttot',
 'yearbuilt',
 'holc_AB',
 'holc_CD',
 'holc_D',
 'tot_hpd_vio',
 'hpd_fire_vio',
 'hpd_b_c_vio',
 'viol_count_per_unit',
 'hpd_fire_vio_per_unit',
 'hpd_b_c_vio_per_unit',
 'AL',
 'NB',
 'average_energy_usage',
 'average_energy_usage_mi',
 'landuse_cat',
 'bldgclass_cat',
 'totpop',
 'popdens',
 'areasqmile',
 'males',
 'females',
 'pop18under',
 'pop18_34',
 'pop35_64',
 'pop65plus',
 'pophispanic',
 'popnhwhite',
 'popnhblack',
 'popnhaian',
 'popnhasian',
 'popnhhawainpi',
 'popnhother',
 'popnhtwoplus',
 'housing',
 'housingoccupied',
 'renterhousing',
 'oc_halfless',
 'oc_half_1',
 'oc_1_onehalf',
 'oc_onehalf_2',
 'oc_2plus',
 'roc_halfless',
 'roc_half_1',
 'roc_1_onehalf',
 'roc_onehalf_2',
 'roc_2plus',
 'medianyearbuilt',
 'mediangrossrent',
 'medianhhincome',
 'gini',
 'year',
 'pct18under',
 'pct18_34',
 'pct35_64',
 'pct65plus',
 'pcthispanic',
 'pctn

In [41]:
X = df.loc[:, ["bldgclass", "landuse", "lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
              "assesstot", "exempttot", "yearbuilt", "holc_AB", "holc_CD", "holc_D", "tot_hpd_vio", "AL", "NB",
              "average_energy_usage", "average_energy_usage_mi"]+borocols] #'yearalter1', 'yearalter2', 

y = df.loc[:, "vacate_ind"]

In [42]:
predictors_fimp=["hpd_b_c_vio", "bldgarea", "tot_hpd_vio", "unitstotal", "pop65plus", "popdens", "gini", 
                 "holc_AB", "holc_CD", "holc_D" ] 
predictors_fimp

['hpd_b_c_vio',
 'bldgarea',
 'tot_hpd_vio',
 'unitstotal',
 'pop65plus',
 'popdens',
 'gini',
 'holc_AB',
 'holc_CD',
 'holc_D']

## Using the Naive Bayes Classifier to predict whether a building had a fire vacate order.

In [43]:
from sklearn.model_selection import train_test_split
#data1_X = data1.iloc[:,1:] # tip, distance, speed, and number of passengers
#data1_y = data1.iloc[:,0] # binary output: 1 if in Manhattan, 0 if outside

X = df.loc[:, predictors_fimp].fillna(0)
y = df.loc[:, "vacate_ind"].fillna(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

In [92]:
# Training a binary Naive Bayes Classifier with discrete input attributes.
# Assume that the binary output variable takes on values 0 or 1. 
def trainNaiveBayes(X,y):
    prior = 1.*y.sum()/y.count()
    nbc = {'prior':prior}
    X_1 = X[y==1]
    X_0 = X[y==0]
    
    for j in X.columns:
        if len(X[j].unique()) <= 5:
            nbc[j+'_1'] = X_1[j].value_counts(normalize=True)
            nbc[j+'_0'] = X_0[j].value_counts(normalize=True)
        else:    
            nbc[j+'_mu1'] = X_1[j].mean()
            nbc[j+'_sigma1'] = X_1[j].std()
            nbc[j+'_mu0'] = X_0[j].mean()
            nbc[j+'_sigma0'] = X_0[j].std()
    return nbc

#mean normal 

def testNaiveBayes(X,nbc):
    y_pred = pd.Series(index=X.index,dtype='float64')
    for i in X.index:
        # compute odds of y=1
        y_pred[i] = nbc['prior']/(1-nbc['prior']) # prior odds
        for j in X.columns:
            if len(X[j].unique()) <= 5: 
                thevalue = X.loc[i,j]
                if thevalue not in nbc[j+'_1']:
                    y_pred[i] = y_pred[i]*1E-3
                if thevalue not in nbc[j+'_0']:
                    y_pred[i] = y_pred[i]*1E3
                if (thevalue in nbc[j+'_1']) & (thevalue in nbc[j+'_0']):
                    y_pred[i] = y_pred[i]*(nbc[j+'_1'][thevalue]+1E-3)/(nbc[j+'_0'][thevalue]+1E-3)
            else:
                thevalue = X.loc[i,j]
                pdf1 = stats.norm.pdf(thevalue,loc=nbc[j+'_mu1'],scale=nbc[j+'_sigma1'])
                pdf0 = stats.norm.pdf(thevalue,loc=nbc[j+'_mu0'],scale=nbc[j+'_sigma0'])
                y_pred[i] = y_pred[i]*pdf1/pdf0 if pdf0 > 0 else 1E10
        # convert odds to probability of y=1
        y_pred[i] = y_pred[i]/(1.0+y_pred[i])
    return y_pred

In [93]:
naive_bayes_classifier = trainNaiveBayes(X_train,y_train)
naive_bayes_classifier;

In [ ]:
naive_bayes_classifier = trainNaiveBayes(X_train,y_train)
for i,j in naive_bayes_classifier.items():
    print(i)
    print(j)
    print()
#y_pred_train = testNaiveBayes(X_train,naive_bayes_classifier)
y_pred_test = testNaiveBayes(X_test,naive_bayes_classifier)
print(y_pred_test)

prior
0.003414266019948795

hpd_b_c_vio_mu1
181.5375191424196

hpd_b_c_vio_sigma1
303.32400833237455

hpd_b_c_vio_mu0
4.232757384445883

hpd_b_c_vio_sigma0
34.08121369728853

bldgarea_mu1
70206.2358346095

bldgarea_sigma1
272583.81216820393

bldgarea_mu0
4817.510947692415

bldgarea_sigma0
33149.18954871002

tot_hpd_vio_mu1
231.8596222562532

tot_hpd_vio_sigma1
384.82686660436167

tot_hpd_vio_mu0
6.327650793095609

tot_hpd_vio_sigma0
43.26016847125735

unitstotal_mu1
75.03981623277183

unitstotal_sigma1
311.3138251007218

unitstotal_mu0
4.669820394886413

unitstotal_sigma0
48.15410893094668

pop65plus_mu1
674.3568147013782

pop65plus_sigma1
452.4926613474534

pop65plus_mu0
629.359546003043

pop65plus_sigma0
392.4057493229459

popdens_mu1
79310.10828381803

popdens_sigma1
40070.06797919319

popdens_mu0
38434.71788643216

popdens_sigma0
27636.48663045158

gini_mu1
0.4843152628892294

gini_sigma1
0.060128413628286935

gini_mu0
0.44427147846302156

gini_sigma0
0.06135656900887913

holc_AB_1

C:\Users\Jennah\AppData\Local\Temp/ipykernel_14592/2125321609.py:40: RuntimeWarning: overflow encountered in double_scalars
  y_pred[i] = y_pred[i]*pdf1/pdf0 if pdf0 > 0 else 1E10
C:\Users\Jennah\AppData\Local\Temp/ipykernel_14592/2125321609.py:42: RuntimeWarning: invalid value encountered in double_scalars
  y_pred[i] = y_pred[i]/(1.0+y_pred[i])


In [90]:
# measure accuracy for the binary prediction task
print('Out of sample prediction accuracy:',1.0*sum((y_pred_test>0.5)==y_test)/len(y_test))

# measure accuracy of the predicted probabilities
print('Log-likelihood (test):',sum(np.log(y_pred_test*y_test+(1-y_pred_test)*(1-y_test))))

Out of sample prediction accuracy: 0.005176281129579571


In [21]:
#Calculate precision and recall
from sklearn.metrics import precision_recall_fscore_support

#Discretize continuous variables

### Try with the full set of variables

In [62]:
X = df.loc[:, predictors].fillna(0)
X.head()

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,tot_hpd_vio,hpd_fire_vio,hpd_b_c_vio,viol_count_per_unit,hpd_fire_vio_per_unit,hpd_b_c_vio_per_unit,AL,NB,average_energy_usage,average_energy_usage_mi
0,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0
1,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,...,11.0,0.0,8.0,1.100000,0.0,0.8,0.0,0.0,0.0,1.0
2,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,56.0,0.0
3,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,...,1.0,0.0,0.0,0.040000,0.0,0.0,0.0,0.0,0.0,1.0
4,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,...,4.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0


In [63]:
y = df.loc[:, "vacate_ind"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

In [64]:
# Training a binary Naive Bayes Classifier with discrete input attributes.
# Assume that the binary output variable takes on values 0 or 1. 
def trainNaiveBayesDiscrete(X,y):
    prior = 1.*y.sum()/y.count()
    nbc = {'prior':prior}
    X_1 = X[y==1]
    X_0 = X[y==0]
    for j in X.columns:
        nbc[j+'_1'] = X_1[j].value_counts(normalize=True)
        nbc[j+'_0'] = X_0[j].value_counts(normalize=True)
    return nbc

def testNaiveBayesDiscrete(X,nbc):
    y_pred = pd.Series(index=X.index,dtype='float64')
    for i in X.index:
        # compute odds of y=1
        y_pred[i] = nbc['prior']/(1-nbc['prior']) # prior odds
        for j in X.columns:
            thevalue = X.loc[i,j]
            if thevalue not in nbc[j+'_1']:
                y_pred[i] = y_pred[i]*1E-3
            if thevalue not in nbc[j+'_0']:
                y_pred[i] = y_pred[i]*1E3
            if (thevalue in nbc[j+'_1']) & (thevalue in nbc[j+'_0']):
                y_pred[i] = y_pred[i]*(nbc[j+'_1'][thevalue]+1E-3)/(nbc[j+'_0'][thevalue]+1E-3)
        # convert odds to probability of y=1
        y_pred[i] = y_pred[i]/(1.0+y_pred[i])
    return y_pred

In [65]:
# measure accuracy for the binary prediction task
print('In sample prediction accuracy:',1.0*sum((y_pred_train>0.5)==y_train)/len(y_train))
print('Out of sample prediction accuracy:',1.0*sum((y_pred_test>0.5)==y_test)/len(y_test))

# measure accuracy of the predicted probabilities
print('Log-likelihood (train):',sum(np.log(y_pred_train*y_train+(1-y_pred_train)*(1-y_train))))
print('Log-likelihood (test):',sum(np.log(y_pred_test*y_test+(1-y_pred_test)*(1-y_test))))

In sample prediction accuracy: 0.9956292380967319
Out of sample prediction accuracy: 0.9903540507758585
Log-likelihood (train): nan
Log-likelihood (test): nan


## Using Gaussian Processes to predict whether a building will have a fire vacate order.

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel,WhiteKernel
np.random.seed(1)

def f(x):
    return x * np.sin(x)

x_obs = np.array([1.,3.,5.,6.,7.,8.]).reshape(-1,1)
y_obs = f(x_obs).ravel()

# Mesh the input space for evaluations of the real function, the prediction and
# its MSE
x_mesh = np.linspace(0,10,1000).reshape(-1,1)
y_mesh = f(x_mesh)

fig = plt.figure(figsize=(10,5))
plt.plot(x_mesh, f(x_mesh), 'r:', label=u'$f(x) = x\,\sin(x)$')
plt.plot(x_obs, y_obs, 'r.', markersize=10, label=u'Observations')
plt.legend()
plt.show()